# Parsing Statistics of Income (SOI) Tax Tables from the IRS
John Mays | maysj@omb.nyc.gov | Created: 03/11/25 | Last Updated: 03/12/25

Data is from the "Individual income tax returns with exemptions and itemized deductions > Publication 1304" category on the [IRS.gov website](https://www.irs.gov/statistics/soi-tax-stats-individual-statistical-tables-by-size-of-adjusted-gross-income).

In [1]:
import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm # cool arabic word here: taqadum (meaning: progress) = تقدم

In [2]:
data_directory = Path('../data')

## Helper Methods:

In [32]:
def find_total_returns_label_cells(sheet:pd.DataFrame) -> list:
    indices = []
    for column in sheet.columns:
        col_matches = sheet[column].str.match(
            r"^taxable[, ]*returns[, ]*total", flags=re.IGNORECASE, na=False
        )
        row_indices = list(sheet.index[col_matches])
        if row_indices:
            indices += [(r, column) for r in row_indices]
    return indices

#### Finding Likely Header Rows:
In the data, the header rows are displayed like: (1) (2) (3)... or perhaps (11) (12) (13)...

These can show up in one of two ways when imported into pandas: as negative or positive values.  But, either way, it is a repeating series of values horizontally decrementing or incrementing by 1, so it should be easy to find.


In [4]:
def find_likely_header_rows(sheet: pd.DataFrame, threshold=5) -> list:
    likely_header_rows = []

    for row_index in sheet.index:
        row = sheet.iloc[row_index, :]
        one_increments = 0
        prev_value = None
        for value in row:
            if isinstance(value, int) and isinstance(prev_value, int):
                if abs(value - prev_value) == 1:
                    one_increments += 1
            prev_value = value
        if one_increments > threshold:
            likely_header_rows.append(row_index)
    return likely_header_rows

In [10]:
def find_tax_year(file_name:str) -> int:
    yy = file_name[0:2]
    cc = '19' if int(yy) >= 90 else '20'
    return int(cc + yy)

In [17]:
def assemble_header(segments:list):
    segments_text_only = [s for s in segments if isinstance(s, str)]
    header = '_'.join(segments_text_only)
    header = re.sub(r"\[|\]|-| |_|\\|\/|//|\n", "_", header) # replace special chars
    header = '_'.join([s for s in header.split('_') if s != '']) # de-duplicate underscores
    header = header.lower()
    return header

In [20]:
def supplement_inadequate_text_headers(num_to_text_headers: dict) -> dict:
    for num_header, text_header in numeric_to_text_headers.items():
        if text_header == 'amount':
            # extract prefix info from the col before it:
            try:
                preceeding_col_index = -(abs(num_header)-1) if num_header < 0 else abs(num_header)-1
                preceeding_col_name = num_to_text_headers[preceeding_col_index]
                search_result = re.search('number', preceeding_col_name)
                if search_result:
                    prefix_end_index = search_result.span()[0]
                    prefix = preceeding_col_name[0:prefix_end_index]
                    num_to_text_headers[num_header] = prefix + text_header
            except KeyError:
                pass
    return num_to_text_headers

## Collecting all of the files into dataframes:

In [5]:
def collect_files(dir: Path) -> dict:
    sheets = {}
    for sheet_path in tqdm([path for path in data_directory.glob('*.xl*')]):
        sheets[sheet_path.name] = pd.read_excel(sheet_path, header=None)
    return sheets

In [6]:
sheets = collect_files(data_directory)

100%|██████████| 2/2 [00:00<00:00,  2.47it/s]


## Parsing the sheets:

In [8]:
logging = True

In [48]:
total_returns_dicts = {}

for sheet_name, sheet in tqdm(sheets.items()):
    tax_year = find_tax_year(sheet_name)
    print(f'Sheet: {sheet_name}, Year: {tax_year}')
    # finding cells with "total returns" -- the value we want
    total_returns_label_cells = find_total_returns_label_cells(sheet)
    # finding likely header rows:
    likely_header_rows = find_likely_header_rows(sheet, threshold=5)
    # finding the numeric values of the total returns:
    indices_and_tr_values = {}
    for total_returns_cell_index in total_returns_label_cells:
        tr_row, tr_column = total_returns_cell_index
        total_returns_cell = sheet.iloc[*total_returns_cell_index]
        possible_numeric_value_indices = [(tr_row, col) for col in sheet.columns if col > tr_column]
        for index in possible_numeric_value_indices:
            value = sheet.iloc[*index]
            if isinstance(value, (int, float)): # then the value is numeric & valid
                indices_and_tr_values[index] = value
    # finding the total returns' corresponding headers:
    numeric_headers_and_values = {}
    for index, value in indices_and_tr_values.items():
        value_row, value_column = index
        # finds the closest header row that is above this value:
        corresponding_header_row = max([row for row in likely_header_rows if row < value_row])
        numeric_header = sheet.iloc[corresponding_header_row, value_column]
        numeric_headers_and_values[numeric_header] = value
    # finding indices for the numeric headers:
    indices_and_numeric_headers = {}
    for header_row_index in likely_header_rows:
        header_row = sheet.iloc[header_row_index, :]
        for potential_header_column, potential_header in zip(header_row.index, header_row):
            if isinstance(potential_header, int) and potential_header < 1000:
                header_index = (header_row_index, potential_header_column)
                indices_and_numeric_headers[header_index] = potential_header
    # associating the numeric to the text headers:
    numeric_to_text_headers = {}
    for header_index, numeric_header in indices_and_numeric_headers.items():
        header_row, header_col = header_index
        lesser_row_index = max(header_row-8, 0)
        greater_row_index = max(header_row, 0)
        text_header_segments = list(sheet.iloc[lesser_row_index:greater_row_index, header_col])
        numeric_to_text_headers[numeric_header] = assemble_header(text_header_segments)

    # patch some of the headers which could only find further title text one cell over:
    numeric_to_text_headers = supplement_inadequate_text_headers(numeric_to_text_headers)

    # Associating values with text headers:
    text_headers_and_values = {}
    all_numeric_headers = set(numeric_headers_and_values.keys()).union(numeric_to_text_headers.keys())
    for numeric_header in all_numeric_headers:
        try:
            text_header = numeric_to_text_headers[numeric_header]
            value = numeric_headers_and_values[numeric_header]
            text_headers_and_values[text_header] = value
        except KeyError:
            pass
    # add tax_year as a header/value pair as well:
    # text_headers_and_values['tax_year'] = tax_year
    total_returns_dicts[tax_year] = text_headers_and_values

100%|██████████| 2/2 [00:00<00:00, 11.96it/s]

Sheet: 22in21id.xls, Year: 2022
Sheet: 99in21id.xls, Year: 1999


In [50]:
total_returns = pd.DataFrame.from_records(total_returns_dicts)

In [51]:
total_returns.head()

,1999,2022
dividends_number_of_returns,16803873.0,NaN
total_income_tax_amount,667631592.0,977279714.0
total_income_tax_number_of_returns,37532174.0,13783612.0
alternative_minimum_tax_amount,5980764.0,2849082.0
alternative_minimum_tax_number_of_returns,920953.0,109595.0
